# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [ ]:
#%%pycodestyle


class MyDecisionTreeClassifier():
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.params = {'min_samples_split': min_samples_split,
                       'max_depth': max_depth,
                       'criterion': criterion,
                       'sufficient_share': sufficient_share,
                       'max_features': max_features}

        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s_1, r_s_1 = l_s.reshape(l_s.shape[0]), r_s.reshape(r_s.shape[0])
        l_part, r_part = (l_s_1 / (l_s_1 + r_s_1)), (r_s_1 / (l_s_1 + r_s_1))
        return ((np.ones(l_s.shape[1]) - np.sum(np.square(l_c / l_s), axis=-1))
                * l_part +
                (np.ones(r_s.shape[1]) - np.sum(np.square(r_c / r_s), axis=-1))
                * r_part)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s_1, r_s_1 = l_s.reshape(l_s.shape[0]), r_s.reshape(r_s.shape[0])
        l_part, r_part = (l_s_1 / (l_s_1 + r_s_1)), (r_s_1 / (l_s_1 + r_s_1))
        p_l, p_r = l_c / l_s, r_c / r_s
        # несмотря на where кидает предупреждение о log(0)
        return ((-np.sum(np.where(p_l == 0, p_l, p_l * np.log(p_l)))) * l_part
                +
                (-np.sum(np.where(p_r == 0, p_r, p_r * np.log(p_r)))) * r_part)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s_1, r_s_1 = l_s.reshape(l_s.shape[0]), r_s.reshape(r_s.shape[0])
        l_part, r_part = (l_s_1 / (l_s_1 + r_s_1)), (r_s_1 / (l_s_1 + r_s_1))
        p_l = np.append(l_c / l_s, np.zeros(l_s.shape), axis=-1)
        p_r = np.append(r_c / r_s, np.zeros(r_s.shape), axis=-1)
        return ((np.ones(l_s.shape[1]) - np.max(p_l, axis=-1)) * l_part +
                (np.ones(r_s.shape[1]) - np.max(p_r, axis=-1)) * r_part)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.array(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:max(1, int(np.ceil(np.sqrt(n_feature))))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.array(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:max(1, int(np.ceil(np.log2(n_feature))))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # - сортирует объекты по фиче x, считает минимальное число объектов
        #   в одной из вершин разбиения
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        # - находит отрезки из одинаковых объектов, сохраняет их границы;
        #   если в центре (без хвостов) все объекты одинаковые,
        #   то разделить нельзя, возвращаем impurity = inf
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        # - считает для каждого разбиения по одному из объектов из r_border_ids
        #   сколько объектов какого класса попадут левее данного объекта
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        # - считает для каждого разбиения по одному из border_ids сколько
        #   объектов какого класса попадут в левую и правую вершины, а также
        #   общее количество объектов в разбиениях
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # - вычисляет impurity каждого разбиения,
        #   находит оптимальное с min impurity
        #
        #   НО делается это в предположении, что каждые два соседних отрезка
        #   можно разбить, однако если два объекта разных классов имеют
        #   одинаковые x, то всё будет плохо, ибо вернётся хороший impurity,
        #   как будто можно разбить, и порогом будет этот одинаковый x,
        #   а когда честно по нему разбиваем в __fit_node, может получиться
        #   разбиение, где один из потомков пустой, или impurity не минимальный
        #   так что надо смотреть ещё и на x, поэтому добавлено ещё условие
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        #####
        gs = gs[sorted_x[l_sizes[:, 0] - 1] != sorted_x[l_sizes[:, 0]]]
        if gs.shape[0] == 0:
            return np.inf, None
        #####
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # - возвращает impurity и границу разбиения
        #   для полученного оптимального разбиения
        left_el_id = l_sizes[idx][0]
        border = (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE     _, feature_id, threshold = node
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node

        features = np.unique(y, return_counts=True)
        shares = features[1] / np.sum(features[1])
        if ((self.max_depth is not None and depth >= self.max_depth) or
                (np.max(shares) >= self.sufficient_share) or
                (y.shape[0] <= self.min_samples_split)):
            feature_id = features[0][np.argmax(shares)]
            node = (self.LEAF_TYPE, feature_id, 0.0)
            self.tree[node_id] = node
            return

        feature_ids = self.get_feature_ids(x.shape[1])

        splits = [(*self.__find_threshold(x[:, f_id], y), f_id)
                  for f_id in feature_ids]
        # нужно сравнивать кортежи целиком
        gs, threshold, feature_id = min(splits)

        if (threshold is None):
            feature_id = features[0][np.argmax(shares)]
            node = (self.LEAF_TYPE, feature_id, 0.0)
            self.tree[node_id] = node
            return

        pred_gs = self.G_function(
            np.array([np.unique(y, return_counts=True)[1]]),
            np.array([[y.shape[0]]]),
            np.array([[]]), np.array([[0]]))

        self.feature_importances_[feature_id] += (pred_gs - gs)

        node = (self.NON_LEAF_TYPE, feature_id, threshold)
        self.tree[node_id] = node

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)
        self.__fit_node(x_l, y_l, node_id * 2 + 1, depth + 1)
        self.__fit_node(x_r, y_r, node_id * 2 + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= np.sum(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def score(self, X_test, y_test):
        return f1_score(y_pred=self.predict(X_test),
                        y_true=y_test, average='macro')

    def get_params(self, deep=True):
        return self.params

    def set_params(self, **params):
        self.__init__(**params)

In [ ]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [ ]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

## Проверка качества работы на wine

In [ ]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

## Подготовка данных Speed Dating Data 

In [ ]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', 
                 encoding='cp1251')


In [ ]:
df = df.iloc[:, :97]

In [ ]:
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)

In [ ]:
df = df.drop(['condtn'], axis=1)

In [ ]:
df = df.drop(['round'], axis=1)

In [ ]:
df = df.drop(['position', 'positin1'], axis=1)

In [ ]:
df = df.drop(['order'], axis=1)

In [ ]:
df = df.drop(['partner'], axis=1)

In [ ]:
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)

In [ ]:
df = df.dropna(subset=['age'])

In [ ]:
df = df.drop(['field'], axis=1)

In [ ]:
df = df.drop(['undergra'], axis=1)

In [ ]:
# Что будем делать?
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

In [ ]:
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)

In [ ]:
# Что будем делать?
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

In [ ]:
# Что делать?

df = df.dropna(subset=['imprelig', 'imprace'])

In [ ]:
df = df.drop(['from', 'zipcode'], axis=1)

In [ ]:
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

In [ ]:
# df = df.drop(['income'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

In [ ]:
df = df.dropna(subset=['date'])

In [ ]:
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)

In [ ]:
df = df.drop(['career'], axis=1)

In [ ]:
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)

In [ ]:
df = df.drop(['expnum'], axis=1)

In [ ]:
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']

In [ ]:
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

In [ ]:
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

In [ ]:
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)

In [ ]:
idx = ((temp.wave >= 6) & (temp.wave <= 9))

In [ ]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)

In [ ]:
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

### Attr2

In [ ]:
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

In [ ]:
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]

In [ ]:
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

In [ ]:
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)

In [ ]:
idx = ((temp.wave >= 6) & (temp.wave <= 9))

In [ ]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)

In [ ]:
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

In [ ]:
df = df.drop(['temp_totalsum'], axis=1)

In [ ]:
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)

In [ ]:
df = df.drop(['wave'], axis=1)

In [ ]:
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

In [ ]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'

In [ ]:
df_female = df_female.drop(['pid_f'], axis=1)

In [ ]:
df_pair = df_male.join(df_female.set_index('iid_f'), 
                       on='pid', 
                       how='inner')

In [ ]:
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

In [ ]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
df_pair.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)

## Проверка скорости работы на Speed Dating Data 

In [ ]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
%time clf.fit(X_train, y_train)

In [ ]:
# best from GridSearchCV
my_clf.set_params(**{'min_samples_split': 7, 'max_features': None, 'max_depth': 10, 'criterion': 'gini'})
# my_clf.set_params(**{'min_samples_split': 7, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini'})
# при случайном везении получилось 0.5999999999 за 114ms

In [ ]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
%time my_clf.fit(X_train, y_train)

## Проверка качества работы на Speed Dating Data

In [ ]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

## Задание 3

In [ ]:
best_clf = sorted(zip(df_pair.keys()[1:], clf.feature_importances_), key = lambda x: -x[1])
print("Best clf features:")
for feature in best_clf[:10]:
    print(feature[0], ':', feature[1])
print()

best_my_clf = sorted(zip(df_pair.keys()[1:], my_clf.feature_importances_), key = lambda x: -x[1])
print("Best my_clf features:")
for feature in best_my_clf[:10]:
    print(feature[0], ':', feature[1])
print()

## Задание 4

In [ ]:
params = {'min_samples_split': list(range(2, 5)), 
          'max_depth': [None] + list(range(3, 6)),
          'criterion': ['gini', 'entropy'], 
          'max_features': [None, 'sqrt', 'log2']}
optimiser = GridSearchCV(DecisionTreeClassifier(), params)
optimiser.fit(X, y)
print(optimiser.best_params_)
print(optimiser.best_score_)

In [ ]:
params = {'min_samples_split': list(range(2, 5)), 
          'max_depth': [None] + list(range(3, 10)),
          'criterion': ['gini', 'entropy', 'misclass'], 
          'max_features': [None, 'sqrt', 'log2']}
# optimiser = GridSearchCV(MyDecisionTreeClassifier(), params,refit=False)
optimiser = RandomisedSearchCV(MyDecisionTreeClassifier(), params,refit=False)
optimiser.fit(X, y)
print(optimiser.best_params_)
print(optimiser.best_score_)